In [ ]:
from requests import get # la vamos a usar para solicitar la página mediante su URL 
from bs4 import BeautifulSoup # biblioteca para web scraping
import pandas as pd
import string                              
from nltk.corpus import stopwords # para remover palabras que no agregan significado          
from nltk.stem import PorterStemmer # para llevar las palabras a su forma raíz        
from nltk.tokenize import TweetTokenizer # para transformar cada palabra en un token
from IPython.display import clear_output
from collections import defaultdict
import numpy as np

def etiquetas(palabras,frecuencias,clases):
    import numpy as np
    x = np.zeros((1, 6))
    for palabra in palabras:
        for t in clases:
            x[0,t] += frecuencias.get((palabra, t),0)
    x[0,0]=1
    return x

stop        = stopwords.words('english')
tokenizer   = TweetTokenizer(preserve_case = False, strip_handles = True, reduce_len = True)
stemmer     = PorterStemmer()
webs        = pd.ExcelFile("WEBs.xls")
df          = webs.parse("Categorias")
textos      = pd.DataFrame(columns = [ 'Texto' , 'Clase'])
clases_df   = set(df["CÓDIGO"].value_counts().index)
frecuencias = defaultdict(int)
for i in df.index:
    url       = df["URL"][i]
    pagina    = get(url)
    html      = BeautifulSoup(pagina.text,"html.parser")
    parrafos  = [ x.text.strip() for x in html.findAll("p") if x.text != "\n" ]
    for p in parrafos:
        palabras = p.split()
        if 15 <= len(palabras):
            clear_output( wait = True )
            print("procesando {}".format(url))
            print(f"{i} de {len(df)}")
            texto_tokens = tokenizer.tokenize(p)
            texto_clean  = [ word for word in texto_tokens if word not in stop and word not in string.punctuation ]
            texto_stem   = [ stemmer.stem(word) for word in texto_clean ]
            for t in range(len(texto_stem)//20):
                j = 20 * t
                palabras  = texto_stem[ j : 20 * (t+1) ] 
                textos   = textos.append({'Texto' : palabras, 'Clase' : df["CÓDIGO"][i] }, ignore_index = True)

vocabulario = set([x for m in textos.index for x in textos.loc[m].Texto ])                
clases_df   = set(df["CÓDIGO"].value_counts().index)

for i in textos.index:
    for palabra in textos.loc[i].Texto:
        frecuencias[(palabra, textos.loc[i].Clase)] += 1
        clear_output( wait = True )
        print(f"Creando Frecuencias {i} de {len(textos)}")

for i in clases_df:
    textos[f"Clase{i}"] = textos.Texto.apply(lambda x : etiquetas(x,frecuencias,clases_df)[0,i])
print("dataset terminado")
print(textos.describe())

In [ ]:
textos

In [ ]:
textos.describe()

In [ ]:
textos.info()

In [ ]:
textos.to_csv("Historia.csv")

In [ ]:
import pandas as pd
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

In [ ]:
textos.groupby(by=["Clase"]).count()

In [ ]:
textos.sample(10)

In [ ]:
len(vocabulario)

In [ ]:
len(frecuencias)

In [ ]:
clases_df

In [ ]:
import pickle
def save_obj(obj, name):
       with open(name + '.pkl', 'wb') as f:
           pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
       with open(name + '.pkl', 'rb') as f:
           return pickle.load(f)

In [ ]:
save_obj(vocabulario, "Vocabulario")

In [ ]:
save_obj(frecuencias, "Frecuencias")

In [ ]:
save_obj(clases_df, "clases_df")

In [ ]:
import numpy as np

In [ ]:
textos.sample(10)

In [ ]:
int(np.argmax(textos.iloc[14317,-5:].values)) + 1

In [ ]:
int(textos.iloc[14317,1:2].values)

In [ ]:
int(np.argmax(textos.iloc[14317,-5:].values)) + 1 == int(textos.iloc[14317,1:2].values)

In [ ]:
for i in textos.index:
    textos[i,"FreqOK"] = int(np.argmax(textos.iloc[i,-6:].values)) == int(textos.iloc[i,1:2].values)

In [ ]:
i = 3
int(np.argmax(textos.iloc[i,-6:].values)) == int(textos.iloc[i,1:2].values)

In [ ]:
int(np.argmax(textos.iloc[i,-5:].values) + 1)

In [ ]:
textos.iloc[i,-6:].values

In [ ]:
textos["Clase1"][0]

In [ ]:
textos["FreqOK"] = False
textos

In [ ]:
for i in textos.index:
    textos["FreqOK"][i] = int(np.argmax(textos.iloc[i,-6:].values)) + 1 == int(textos.iloc[i,1:2].values)
textos    

In [ ]:
textos.sample(10)

In [ ]:
textos = textos.sort_values(by=['FreqOK'])


In [ ]:
textos

In [ ]:
textos.groupby(by=["FreqOK"]).count()

In [ ]:
textosF = textos[textos['FreqOK'] == False]
textosF

In [ ]:
textosF.groupby(by=["Clase"]).count()

In [ ]:
textosF1 = textosF[textosF['Clase'] == 1]
textosF1

In [ ]:
claseMax = np.argmax(textosF1.iloc[0,-6:-1].values) + 1
claseMax

In [ ]:
textosF1["ClasMax"] = None
textosF1

In [ ]:
indice = 0
for i in textosF1.index:
    textosF1["ClasMax"][i] = np.argmax(textosF1.iloc[indice,-7:-2].values) + 1
    indice +=1
textosF1.head(11)    

In [ ]:
textosF1.sample(10)    

In [ ]:
frecuencias = load_obj("Frecuencias")

In [ ]:
vocabulario = load_obj("Vocabulario")

In [ ]:
claseTrue = 1
clases = 5
palabrasConfusas = []
# indice = 0
totalPalabras = 0
for i in textosF1.index:
    """
    if indice > 1:
        break"""
    
    w = 0        
    for word in textosF1.loc[i,"Texto"].split():
        freqWord = []
        for k in range(clases + 1):
            freqWord.append(0)
        if w == 0:
            word = word[2:-2]
        else:   
            word = word[1:-2]
        w += 1                  
        if word in vocabulario:
            for k in range(1, len(freqWord)):
                print(word,"clase ",k,frecuencias[(word,k)])
                freqWord[k] = frecuencias[(word,k)]
        print(freqWord)    
        print(np.argmax(freqWord))
        print(np.argmax(freqWord) == claseTrue)        
        if np.argmax(freqWord) != claseTrue:
            palabrasConfusas.append(word)
        totalPalabras += w        
    indice += 1
print("palabras",totalPalabras)
print("confusas",len(palabrasConfusas))
palabrasConfusas

In [ ]:
claseTrue = 1
clases = 5

for word in palabrasConfusas:
    freqWord = []
    for k in range(clases + 1):
        freqWord.append(0)
    if word in vocabulario:
        print()                                  
        for i in range(1,6):
            print(word,"clase ",i,frecuencias[(word,i)]) 
            freqWord[i] = frecuencias[(word,i)]
        print(freqWord)    
        print(np.argmax(freqWord))
        print(np.argmax(freqWord) == claseTrue)
palabrasConfusas        
              

In [ ]:
if 'bronz' in vocabulario:
        print("está")

In [ ]:
print(frecuencias)

In [ ]:
len(palabrasConfusas)

In [ ]:
for word in palabrasConfusas:
    if word in vocabulario:
        print()                                  
        for i in range(1,6):
            print(word,"clase ",i,frecuencias[(word,i)]) 
            freqWord[i] = frecuencias[(word,i)]
        print(freqWord)    
        print(np.argmax(freqWord))
        print(np.argmax(freqWord) == claseTrue)
        print("Maximo", freqWord[np.argmax(freqWord)],"Clase",np.argmax(freqWord))
palabrasConfusas        

In [ ]:
confusas = pd.DataFrame(columns = ['Palabra' , 'Clase','FrecClase','ClaseMax','FrecMax'])

In [ ]:
for word in palabrasConfusas:
    if word in vocabulario: 
        freqWord = []
        for k in range(clases + 1):
            freqWord.append(0)
        for i in range(1,6):
            freqWord[i] = frecuencias[(word,i)]
        confusas = confusas.append({'Palabra' : word, 'Clase' : claseTrue ,'FrecClase' : freqWord[claseTrue], 'ClaseMax' : np.argmax(freqWord),'FrecMax' : freqWord[np.argmax(freqWord)] }, ignore_index = True)        
confusas

In [ ]:
confusas.to_csv("Palabras_Confusas.csv")

In [10]:
import pandas as pd
confusas = pd.read_csv("Palabras_Confusas.csv")
confusas

,Unnamed: 0,Palabra,Clase,FrecClase,ClaseMax,FrecMax
0,0,late,1,66,5,95
1,1,bronz,1,36,5,196
2,2,age,1,83,5,302
3,3,1200,1,3,5,6
4,4,follow,1,105,3,120
...,...,...,...,...,...,...
12366,12366,v,1,10,3,39
12367,12367,larg,1,92,5,159
12368,12368,armi,1,125,4,269
12369,12369,end,1,73,4,157


In [11]:
confusas = confusas.iloc[:,1:]
confusas

,Palabra,Clase,FrecClase,ClaseMax,FrecMax
0,late,1,66,5,95
1,bronz,1,36,5,196
2,age,1,83,5,302
3,1200,1,3,5,6
4,follow,1,105,3,120
...,...,...,...,...,...
12366,v,1,10,3,39
12367,larg,1,92,5,159
12368,armi,1,125,4,269
12369,end,1,73,4,157


In [12]:
confusas["Porcentaje"] = 100 * confusas["FrecClase"] / confusas["FrecMax"]
confusas

,Palabra,Clase,FrecClase,ClaseMax,FrecMax,Porcentaje
0,late,1,66,5,95,69.473684
1,bronz,1,36,5,196,18.367347
2,age,1,83,5,302,27.483444
3,1200,1,3,5,6,50.000000
4,follow,1,105,3,120,87.500000
...,...,...,...,...,...,...
12366,v,1,10,3,39,25.641026
12367,larg,1,92,5,159,57.861635
12368,armi,1,125,4,269,46.468401
12369,end,1,73,4,157,46.496815


In [13]:
confusas["Decenas"] = confusas["Porcentaje"] // 10
confusas

,Palabra,Clase,FrecClase,ClaseMax,FrecMax,Porcentaje,Decenas
0,late,1,66,5,95,69.473684,6.0
1,bronz,1,36,5,196,18.367347,1.0
2,age,1,83,5,302,27.483444,2.0
3,1200,1,3,5,6,50.000000,5.0
4,follow,1,105,3,120,87.500000,8.0
...,...,...,...,...,...,...,...
12366,v,1,10,3,39,25.641026,2.0
12367,larg,1,92,5,159,57.861635,5.0
12368,armi,1,125,4,269,46.468401,4.0
12369,end,1,73,4,157,46.496815,4.0


In [15]:
confusas.groupby(by=["Decenas"]).count()

,Palabra,Clase,FrecClase,ClaseMax,FrecMax,Porcentaje
Decenas,,,,,,
0.0,354,354,354,354,354,354
1.0,605,605,605,605,605,605
2.0,1098,1098,1098,1098,1098,1098
3.0,1365,1365,1365,1365,1365,1365
4.0,1586,1586,1586,1586,1586,1586
5.0,1863,1863,1863,1863,1863,1863
6.0,1926,1926,1926,1926,1926,1926
7.0,1605,1605,1605,1605,1605,1605
8.0,1349,1349,1349,1349,1349,1349
